In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
data = pd.read_csv('../input/titanic/train.csv')
X_test = pd.read_csv('../input/titanic/test.csv')

In [3]:
y = data.Survived

In [4]:
x1 = data.drop('Survived', axis = 1)

In [5]:
x2 = x1.drop('Cabin', axis = 1)

In [6]:
x2.Embarked.fillna("C", inplace = True)

In [7]:
x3 = x2.set_index(x2.columns[0])

In [8]:
x3.drop('Ticket', axis = 1)

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,23.4500,S


In [9]:
l = []
for i in x3['Name']:
    i = i.split('.')[0].split(',')[1]
    l.append(i)
    
x3['Title'] = l 

In [10]:
x3.Title

PassengerId
1         Mr
2        Mrs
3       Miss
4        Mrs
5         Mr
       ...  
887      Rev
888     Miss
889     Miss
890       Mr
891       Mr
Name: Title, Length: 891, dtype: object

In [11]:
x3['Title'].value_counts()

 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Major             2
 Col               2
 Mlle              2
 Jonkheer          1
 Capt              1
 Sir               1
 Don               1
 Ms                1
 Lady              1
 Mme               1
 the Countess      1
Name: Title, dtype: int64

In [12]:
x3['Title'] = x3['Title'].replace([' Don', ' Rev', ' Dr', ' Mme',' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
                                   ' the Countess', ' Jonkheer'], 'Rare')

In [13]:
x4 = x3.drop('Name', axis = 1)
x5 = x4.drop('Ticket', axis = 1)

In [14]:
age_ref = pd.DataFrame([x5.groupby('Pclass').Age.mean()],columns=x5['Pclass'].unique())
age_ref

,3,1,2
Age,25.14062,38.233441,29.87763


In [15]:
def fill_age(pclass,age):
    if pd.isnull(age):
        return float(age_ref[pclass])
    else:
        return age

x5['Age'] = x5.apply(lambda x: fill_age(x['Pclass'],x['Age']), axis=1)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x5, y, test_size = 0.4, random_state = 4)

In [17]:
from sklearn.preprocessing import LabelEncoder

label_X_train = X_train.copy()
label_X_test = X_test.copy()

s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_test[col] = label_encoder.transform(X_test[col])

In [18]:
label_X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,
386,2,1,18.000000,0,0,73.5000,2,2
178,1,0,50.000000,0,0,28.7125,0,1
203,3,1,34.000000,0,0,6.4958,2,2
796,2,1,39.000000,0,0,13.0000,2,2
816,1,1,38.233441,0,0,0.0000,2,2
...,...,...,...,...,...,...,...,...
361,3,1,40.000000,1,4,27.9000,2,2
710,3,1,25.140620,1,1,15.2458,0,0
440,2,1,31.000000,0,0,10.5000,2,2


In [19]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(label_X_train, y_train)

y_pred = logreg.predict(label_X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [20]:
from sklearn import metrics 
print(metrics.accuracy_score(y_test, y_pred))

0.834733893557423


In [21]:
X_real = pd.read_csv('../input/titanic/test.csv')

In [22]:
X_real

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [23]:

X_real = X_real.set_index(X_real.columns[0])
X_real = X_real.drop('Cabin', axis=1)
X_real.Embarked.fillna("C", inplace= True)

m = []
for i in X_real['Name']:
    i = i.split('.')[0].split(',')[1]
    m.append(i)
    
X_real['Title'] = m

X_real['Title'] = X_real['Title'].replace([' Don', ' Rev', ' Dr', ' Mme',' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
                                   ' the Countess', ' Jonkheer', 'Dona'], 'Rare')
X_real = X_real.drop('Name', axis = 1)
X_real = X_real.drop('Ticket', axis = 1)

In [24]:
age_ref = pd.DataFrame([X_real.groupby('Pclass').Age.mean()],columns=X_real['Pclass'].unique())
age_ref

,3,2,1
Age,24.027945,28.7775,40.918367


In [25]:
X_real['Age'] = X_real.apply(lambda x: fill_age(x['Pclass'],x['Age']), axis=1)

In [26]:
X_real

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,
892,3,male,34.500000,0,0,7.8292,Q,Mr
893,3,female,47.000000,1,0,7.0000,S,Mrs
894,2,male,62.000000,0,0,9.6875,Q,Mr
895,3,male,27.000000,0,0,8.6625,S,Mr
896,3,female,22.000000,1,1,12.2875,S,Mrs
...,...,...,...,...,...,...,...,...
1305,3,male,24.027945,0,0,8.0500,S,Mr
1306,1,female,39.000000,0,0,108.9000,C,Dona
1307,3,male,38.500000,0,0,7.2500,S,Mr


In [27]:
X_real.groupby('Embarked').Fare.mean()

Embarked
C    66.259765
Q    10.957700
S    28.230436
Name: Fare, dtype: float64

In [28]:
X_real.Fare.fillna(28.230436, inplace= True)

In [29]:
X_real.Title.unique()

array([' Mr', ' Mrs', ' Miss', ' Master', 'Rare', ' Dona'], dtype=object)

In [30]:
X_real['Title'] = X_real['Title'].replace([' Dona'], 'Rare')

In [31]:
label_X_real = X_real.copy()

label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_real[col] = label_encoder.transform(X_real[col])

In [32]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(label_X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [33]:
label_X_real.fillna(label_X_real.Age.mean(), inplace= True)

In [34]:
final_ans = forest_model.predict(label_X_real)

In [35]:
final_ans

array([0.27102381, 0.38      , 0.57      , 0.15      , 0.76      ,
       0.09      , 0.17      , 0.35      , 0.89      , 0.2       ,
       0.        , 0.11      , 1.        , 0.38      , 1.        ,
       0.97      , 0.12961905, 0.58      , 0.63      , 0.47      ,
       0.79      , 0.86      , 1.        , 0.1335    , 0.89      ,
       0.02      , 1.        , 0.61      , 0.68      , 0.13      ,
       0.14      , 0.158     , 0.69      , 0.27      , 0.8       ,
       0.05      , 0.146     , 0.21      , 0.01      , 0.34104762,
       0.25333333, 0.35      , 0.22      , 1.        , 1.        ,
       0.008     , 0.63      , 0.        , 0.94      , 0.82      ,
       0.393     , 0.34      , 0.9       , 0.96      , 0.36      ,
       0.02      , 0.03285714, 0.        , 0.02      , 1.        ,
       0.11      , 0.24661905, 0.04      , 0.61875   , 0.63      ,
       0.97      , 0.79      , 0.07      , 0.28      , 0.87      ,
       0.80859127, 0.02      , 0.2       , 0.53      , 1.     

In [36]:
l = []
for i in final_ans:
    if i >= 0.5:
        l.append(1)
    elif i< 0.5:
        l.append(0)

In [37]:
l

[0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,


In [38]:
output = pd.DataFrame({'PassengerId': X_real.index,
                       'Survived': l})
output.to_csv('submission1.csv', index=False)